In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [ ]:
!pip install transformers

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
predict_step(['/content/WhatsApp Image 2023-04-24 at 12.28.40 AM.jpeg'])

['a woman in a pink dress is in front of a window']

In [ ]:
!pip install gradio



In [ ]:
!pip install pillow
!pip install torch

In [ ]:
import gradio as gr

# Define the function that uses your image captioning model
def generate_caption(image):
    image_path = "temp_image.jpg"
    image.save(image_path)
    captions = predict_step([image_path])
    return captions[0]

# Create the Gradio interface
image_input = gr.inputs.Image(label="Input Image")
caption_output = gr.outputs.Textbox(label="Generated Caption")

gr.Interface(fn=generate_caption, inputs=image_input, outputs=caption_output, title="Image Captioning", server_name="0.0.0.0").launch()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
<ipython-input-11-a456cfec3d45>:14: UserWarning: `server_name` is deprecated in `Interface()`, please use it within `launch()` instead.
  gr.Interface(fn=generate_caption, inputs=image_input, outputs=caption_output, title="Image Captioning", server_name="0.0.0.0").launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>